In [1]:
from pathlib import Path

In [2]:


input_files = Path("/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.shortcut-targets-by-id/1u0tmSqWpyjE3etisjtWQ83r3cS2LEk_i/Manual Counting /Drone Deploy orthomosaics").rglob("Snt_STJB*.tif")
input_files = list(input_files)
output_dir = Path("/Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile")

input_files

[PosixPath('/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.shortcut-targets-by-id/1u0tmSqWpyjE3etisjtWQ83r3cS2LEk_i/Manual Counting /Drone Deploy orthomosaics/Snt_STJBB01-02_10012023.tif'),
 PosixPath('/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.shortcut-targets-by-id/1u0tmSqWpyjE3etisjtWQ83r3cS2LEk_i/Manual Counting /Drone Deploy orthomosaics/Snt_STJB01to05_10012023.tif'),
 PosixPath('/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.shortcut-targets-by-id/1u0tmSqWpyjE3etisjtWQ83r3cS2LEk_i/Manual Counting /Drone Deploy orthomosaics/Snt_STJB01to05_10012023-1.tif'),
 PosixPath('/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.shortcut-targets-by-id/1u0tmSqWpyjE3etisjtWQ83r3cS2LEk_i/Manual Counting /Drone Deploy orthomosaics/Snt_STJB01to05_10012023-2.tif'),
 PosixPath('/Users/christian/Library/CloudStorage/GoogleDrive-christian.winkelmann@gmail.com/.sh

In [3]:
from loguru import logger
import rasterio
from rasterio.windows import Window
from pathlib import Path
from rasterio.enums import Resampling

def create_tiles(input_file, output_dir, tile_size=512):
    """Split a large raster into smaller physical tiles."""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    cog_options = {
        "BLOCKXSIZE": 1024,  # Internal tile size for COGs
        "BLOCKYSIZE": 1024,
        "TILED": True,
        "COMPRESS": "LZW",  # Compression for COGs
    }
    
    with rasterio.open(input_file) as src:
        width, height = src.width, src.height
        for i in range(0, width, tile_size):
            for j in range(0, height, tile_size):
                # Define window for the tile
                window = Window(i, j, tile_size, tile_size)
                transform = src.window_transform(window)
                
                # Read data and write to a new file
                tile_file = output_dir / f"{input_file.stem}_tile_{i}_{j}.tif"
                meta = src.meta.copy()
                meta.update({
                    "driver": "COG",  # Save directly as COG
                    "height": min(tile_size, height - j),
                    "width": min(tile_size, width - i),
                    "transform": transform,
                    **cog_options,  # Add COG-specific options
                })

                with rasterio.open(tile_file, "w", **meta) as dst:
                    logger.info(f"Writing {tile_file}")
                    dst.write(src.read(window=window))
                    
                    logger.info(f"Adding overviews to {tile_file}")
                    # Add overviews
                    overviews = [2, 4, 8]  # Define overview levels
                    dst.build_overviews(overviews, Resampling.average)
                    dst.update_tags(ns="rio_overview", resampling="average")
                logger.info(f"Saved {tile_file}")

# Example Usage


In [ ]:
for input_file in input_files:
    create_tiles(input_file, output_dir / input_file.stem, tile_size=10240)
    
    

2024-11-22 12:53:10.491 | INFO     | __main__:create_tiles:39 - Writing /Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile/Snt_STJBB01-02_10012023/Snt_STJBB01-02_10012023_tile_0_0.tif
2024-11-22 12:53:11.353 | INFO     | __main__:create_tiles:42 - Adding overviews to /Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile/Snt_STJBB01-02_10012023/Snt_STJBB01-02_10012023_tile_0_0.tif
2024-11-22 12:53:16.390 | INFO     | __main__:create_tiles:47 - Saved /Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile/Snt_STJBB01-02_10012023/Snt_STJBB01-02_10012023_tile_0_0.tif
2024-11-22 12:53:16.391 | INFO     | __main__:create_tiles:39 - Writing /Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile/Snt_STJBB01-02_10012023/Snt_STJBB01-02_10012023_tile_0_10240.tif
2024-11-22 12:53:17.163 | INFO     | __main__:create_tiles:42 - Adding overviews to /Users/christian/PycharmProjects/hnee/image_template_search/data/cog/tile/